# Bubble Sort — Code-level Tracer (Jupyter)

This notebook shows a visual, code-level trace of Bubble Sort:
- Left: highlighted source code line that is executing
- Right: bar chart of the array state
- Controls: Generate, Play, Step, Reset, Size, Speed

Requirements:
- ipywidgets, matplotlib, numpy

Run the next cell to install missing packages (if needed), then run the visualization cell.

In [ ]:
# Install dependencies if you don't have them. Uncomment and run if needed.
# Note: you may need to restart the kernel after installing ipywidgets.
# !pip install ipywidgets matplotlib numpy


In [ ]:
import textwrap
import random
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
import ipywidgets as widgets

# Source displayed on the left (keeps line numbers stable)
SOURCE = textwrap.dedent("""
def bubble_sort(a):
    n = len(a)
    if n <= 1:
        return a
    for passnum in range(n - 1):
        made_swap = False
        # inner loop: compare adjacent pairs
        for j in range(n - passnum - 1):
            # compare a[j] and a[j+1]
            if a[j] > a[j+1]:
                # swap them
                a[j], a[j+1] = a[j+1], a[j]
                made_swap = True
        # optional early exit
        if not made_swap:
            break
    return a
""").strip().splitlines()

def bubble_sort_trace(arr):
    """
    Yields trace frames while executing bubble sort on a copy of arr.
    Each frame is a dict with:
      action: 'enter'|'compare'|'swap'|'pass_end'|'done'
      line: index in SOURCE to highlight (0-based)
      passnum, i, j, array: contextual data
    """
    a = arr[:]
    n = len(a)
    # initial
    yield dict(action='enter', line=0, passnum=-1, i=-1, j=-1, array=a.copy())
    yield dict(action='enter', line=1, passnum=-1, i=-1, j=-1, array=a.copy())
    if n <= 1:
        yield dict(action='done', line=2, passnum=-1, i=-1, j=-1, array=a.copy())
        return
    for passnum in range(n - 1):
        yield dict(action='enter', line=3, passnum=passnum, i=-1, j=-1, array=a.copy())
        made_swap = False
        yield dict(action='enter', line=4, passnum=passnum, i=-1, j=-1, array=a.copy())
        for j in range(n - passnum - 1):
            yield dict(action='compare', line=6, passnum=passnum, i=j, j=j+1, array=a.copy())
            if a[j] > a[j + 1]:
                a[j], a[j + 1] = a[j + 1], a[j]
                made_swap = True
                yield dict(action='swap', line=8, passnum=passnum, i=j, j=j+1, array=a.copy())
        yield dict(action='pass_end', line=10, passnum=passnum, i=-1, j=-1, array=a.copy())
        if not made_swap:
            break
    yield dict(action='done', line=11, passnum=passnum if n>1 else -1, i=-1, j=-1, array=a.copy())

def random_array(n):
    return [random.randint(1, max(1, n*4)) for _ in range(n)]

class BubbleSortNotebookVisualizer:
    def __init__(self, n=18, speed=150):
        self.n = n
        self.speed = int(speed)
        self.arr = random_array(self.n)
        self.original = self.arr[:]
        self.steps = list(bubble_sort_trace(self.arr))
        self.frame = 0

        # Matplotlib figure
        self.fig, self.ax = plt.subplots(figsize=(8,4))
        self.bars = self.ax.bar(range(len(self.arr)), self.arr, color='#4f83ff', edgecolor='black')
        self.ax.set_ylim(0, max(self.arr) + max(1, int(max(self.arr)*0.05)))
        self.ax.set_title("Array state")

        # Create code HTML area
        self.code_html = widgets.HTML(value=self._make_code_html(highlight_line=None), 
                                      layout=widgets.Layout(width='48%', height='360px', overflow='auto'))
        # Display area for matplotlib; use an Output widget
        self.plot_out = widgets.Output(layout=widgets.Layout(width='52%'))
        with self.plot_out:
            display(self.fig)
        plt.close(self.fig)

        # Status text
        self.status = widgets.HTML(value=self._status_html("idle", 0, 0, '-'))

        # Controls
        self.btn_generate = widgets.Button(description='Generate', tooltip='Generate new array')
        self.btn_play = widgets.ToggleButton(description='Play', tooltip='Play / Pause')
        self.btn_step = widgets.Button(description='Step', tooltip='Advance one step')
        self.btn_reset = widgets.Button(description='Reset', tooltip='Reset to initial array')
        self.size_slider = widgets.IntSlider(value=self.n, min=5, max=120, description='Size')
        self.speed_slider = widgets.IntSlider(value=self.speed, min=10, max=1000, step=10, description='Speed (ms)')
        self.frame_slider = widgets.IntSlider(value=0, min=0, max=max(0, len(self.steps)-1), description='Frame', continuous_update=True)
        self.play_widget = widgets.Play(value=0, min=0, max=max(0, len(self.steps)-1), step=1, interval=self.speed)
        widgets.jslink((self.play_widget, 'value'), (self.frame_slider, 'value'))

        # Hook events
        self.btn_generate.on_click(self._on_generate)
        self.btn_step.on_click(self._on_step)
        self.btn_reset.on_click(self._on_reset)
        self.btn_play.observe(self._on_play_toggle, names='value')
        self.size_slider.observe(self._on_size_change, names='value')
        self.speed_slider.observe(self._on_speed_change, names='value')
        self.frame_slider.observe(self._on_frame_change, names='value')

        # Layout
        controls = widgets.HBox([
            self.btn_generate, self.btn_play, self.btn_step, self.btn_reset,
            self.size_slider, self.speed_slider
        ], layout=widgets.Layout(align_items='center', flex_flow='row wrap'))
        top = widgets.HBox([self.code_html, self.plot_out])
        bottom = widgets.HBox([self.frame_slider, self.play_widget, self.status], layout=widgets.Layout(align_items='center', justify_content='space-between'))
        self.ui = widgets.VBox([controls, top, bottom])
        # initial render
        self._render_frame(0)

    def _make_code_html(self, highlight_line=None):
        lines = []
        for idx, line in enumerate(SOURCE):
            safe = (line.replace('&','&amp;').replace('<','&lt;').replace('>','&gt;'))
            num = f"{idx+1:>2}"
            if idx == highlight_line:
                lines.append(f'<div style="background:gold;padding:2px 6px;border-radius:4px;"><code style="font-family:monospace;">{num}: {safe}</code></div>')
            else:
                lines.append(f'<div><code style="font-family:monospace;color:#dfefff;">{num}: {safe}</code></div>')
        return "<div style='background:#071127;padding:10px;border-radius:8px;color:#e6eef8;'>" + "\n".join(lines) + "</div>"

    def _status_html(self, op, comps, swaps, passnum):
        return f"<div style='font-family:monospace;'><b>op:</b> {op} &nbsp;&nbsp; <b>comparisons:</b> {comps} &nbsp;&nbsp; <b>swaps:</b> {swaps} &nbsp;&nbsp; <b>pass:</b> {passnum}</div>"

    def _on_generate(self, b):
        self.n = int(self.size_slider.value)
        self.arr = random_array(self.n)
        self.original = self.arr[:]
        self.steps = list(bubble_sort_trace(self.arr))
        self.frame = 0
        self._reset_widgets_after_steps()
        self._render_frame(0)

    def _on_size_change(self, change):
        self._on_generate(None)

    def _on_speed_change(self, change):
        self.speed = int(self.speed_slider.value)
        self.play_widget.interval = self.speed

    def _on_step(self, b):
        self.btn_play.value = False
        if self.frame_slider.value < len(self.steps) - 1:
            self.frame_slider.value = self.frame_slider.value + 1
        else:
            self.frame_slider.value = len(self.steps) - 1

    def _on_reset(self, b):
        self.btn_play.value = False
        self.arr = self.original[:]
        self.steps = list(bubble_sort_trace(self.arr))
        self.frame_slider.max = max(0, len(self.steps)-1)
        self.play_widget.max = self.frame_slider.max
        self.frame_slider.value = 0
        self._render_frame(0)

    def _on_play_toggle(self, change):
        if change['new']:
            self.play_widget.max = max(0, len(self.steps)-1)
            self.play_widget.value = self.frame_slider.value
            self.btn_play.description = 'Pause'
        else:
            self.btn_play.description = 'Play'
            self.play_widget.value = self.frame_slider.value

    def _on_frame_change(self, change):
        new_frame = int(change['new'])
        self.frame = new_frame
        self._render_frame(new_frame)
        if new_frame >= len(self.steps) - 1:
            self.btn_play.value = False

    def _reset_widgets_after_steps(self):
        self.frame_slider.max = max(0, len(self.steps)-1)
        self.play_widget.max = self.frame_slider.max
        self.play_widget.value = 0
        self.frame_slider.value = 0

    def _render_frame(self, idx):
        if not (0 <= idx < len(self.steps)):
            return
        step = self.steps[idx]
        arr_snap = step['array']
        action = step['action']
        line = step['line']
        passnum = step.get('passnum', '-')
        i = step.get('i', -1)
        j = step.get('j', -1)

        # Update bars
        ax = self.ax
        ax.clear()
        colors = ['#4f83ff'] * len(arr_snap)
        if action == 'compare':
            if 0 <= i < len(colors): colors[i] = 'gold'
            if 0 <= j < len(colors): colors[j] = 'gold'
        elif action == 'swap':
            if 0 <= i < len(colors): colors[i] = 'crimson'
            if 0 <= j < len(colors): colors[j] = 'crimson'
        elif action == 'pass_end':
            tail_start = len(arr_snap) - (passnum + 1)
            for k in range(tail_start, len(arr_snap)):
                if 0 <= k < len(colors):
                    colors[k] = 'seagreen'
        elif action == 'done':
            colors = ['seagreen'] * len(colors)

        self.bars = ax.bar(range(len(arr_snap)), arr_snap, color=colors, edgecolor='black')
        ax.set_ylim(0, max(arr_snap) + max(1, int(max(arr_snap)*0.05)))
        ax.set_title("Array state")

        # Update code HTML (highlight the specified line)
        self.code_html.value = self._make_code_html(highlight_line=line)

        # Update status counts by scanning up to idx
        comps = sum(1 for k in range(idx+1) if self.steps[k]['action'] == 'compare')
        swaps = sum(1 for k in range(idx+1) if self.steps[k]['action'] == 'swap')
        self.status.value = self._status_html(action, comps, swaps, passnum)

        # Force refresh of plot in the output widget
        with self.plot_out:
            display(self.fig)

    def show(self):
        display(self.ui)

# Create and show visualizer (default size 18, speed 150ms)
visualizer = BubbleSortNotebookVisualizer(n=18, speed=150)
visualizer.show()
